In [17]:
import os
from datetime import datetime
from keras.models import *
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten, Dense, UpSampling2D
from keras.models import *
from keras.optimizers import SGD,Adam,Adagrad,Nadam,RMSprop
# from keras.losses import binary_crossentropy
# from keras.metrics import binary_accuracy
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

### Create directories

In [18]:

def create_folder(_fold_path):
    try:
        os.makedirs(_fold_path)
    except OSError:
        print("***folder already exist***")

### define network

In [19]:

def simpleCNN(input_shape=(48, 48, 3), classes=1):
    img_input = Input(shape=input_shape)
    bn_axis = 3

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(img_input, decoded)
    return model

### Training model

In [20]:
# train model
def train_model(model, loss, metrics, optimizer, epochs=25):
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    model.summary()
    history = model.fit_generator(
        train_generator,
        # steps_per_epoch=num_train_samples // batch_size,
        
        epochs=epochs,
        callbacks=callbacks,
        validation_data=val_generator,
        class_weight='auto',
        steps_per_epoch=50,
        validation_steps=30,
        # validation_steps=num_val_samples // batch_size,
    )
    return model


In [21]:
if __name__ == '__main__':
    # dimensions of our images.
    IM_HEIGHT, IM_WIDTH, IM_CHANNEL = 32, 32, 3
    NB_EPOCHS = 20
    BAT_SIZE = 16
    FC_SIZE = 1024
    # data path
    _data_path = './asset/data/'
    train_data_dir = _data_path+"train/"
    valid_data_dir = _data_path+"valid/"
    test_data_dir = _data_path+"test/"

    train_datagen = ImageDataGenerator(
        rescale=1. / 255,   # orignal image is 0~255
     
        )

In [22]:
 # augmentation configuration use for validation and testing only rescaling
val_datagen = ImageDataGenerator(
        rescale=1. / 255,   
    )

In [23]:
test_datagen = ImageDataGenerator(
        rescale=1. / 255,   
    )

In [24]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(IM_HEIGHT, IM_WIDTH), # resize the images
        batch_size=BAT_SIZE,
        class_mode="input"
    )

Found 254 images belonging to 2 classes.


In [25]:
val_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(IM_HEIGHT, IM_WIDTH),  # resize the images
        batch_size=BAT_SIZE,
        class_mode="input",
        shuffle=False
    )

Found 10 images belonging to 1 classes.


In [26]:
# the samples and classes generated by the generator 
num_train_samples = train_generator.samples
num_val_samples = val_generator.samples
num_classes = train_generator.num_classes


ts = datetime.now().strftime("%Y-%m%d-%H%M")
_log_dir = './logs/log-{ts}/'.format(ts=ts)
    
if not os.path.exists(_log_dir):
    os.makedirs(_log_dir)
tensorboard = TensorBoard(log_dir=_log_dir)

callbacks = []
callbacks.append(tensorboard)

save_dir = './models/save-{ts}/'.format(ts=ts)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
filepath= save_dir+"improve-{epoch:02d}-{val_acc:.2f}.h5"
    
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, mode='max')
callbacks.append(checkpoint)


In [27]:

# To initialize the model
model = simpleCNN(input_shape=(IM_HEIGHT,IM_WIDTH,IM_CHANNEL), classes=num_classes)
# The methods of optimization
opt = Nadam(lr = 0.002)

model = train_model(model, loss='binary_crossentropy', metrics=['accuracy'], optimizer='adadelta', epochs=NB_EPOCHS)

print("\n #################### Save last model ###################### \n")
ts = datetime.now().strftime("%Y-%m%d-%H%M")
model.save(save_dir+'model-{ts}.h5'.format(ts=ts))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 32)        18464     
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 16)          4624      
__________

KeyboardInterrupt: 

## Visualize original image and reconstructed image

In [ ]:
import matplotlib.pyplot as plt
import cv2
import matplotlib
decoded_imgs = model.predict_generator(val_generator,steps=1)
train_data_dir_female=train_data_dir+"female/"
n=10
plt.figure(figsize=(10,4))
for i in range(n):
    # display original
    # ax = plt.subplot(2, n, i+1)
    #plt.imshow(decoded_imgs[i].reshape(32, 32,3))
    #plt.gray()
    #ax.get_xaxis().set_visible(False)
    #ax.get_yaxis().set_visible(False)
    # display restruction 
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(32,32,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False) 
    matplotlib.image.imsave(train_data_dir_female+str(i)+'.jpg', decoded_imgs[i].reshape(32,32,3))
plt.show()

In [ ]:
import glob
import matplotlib.pyplot as plt

img_path = glob.glob("./asset/data/test/test_image/*.jpg")
i=0;
plt.figure(figsize=(10,4))

for img in img_path:
    a=plt.imread(img)
    i=i+1;
    ax = plt.subplot(4, 10,i)
    plt.imshow(a)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False) 
    
plt.show()

# Exercise  1 Training a model with generated images

In [ ]:
import os
from datetime import datetime
from keras.models import *
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten, Dense, UpSampling2D
from keras.models import *
from keras.optimizers import SGD,Adam,Adagrad,Nadam,RMSprop
# from keras.losses import binary_crossentropy
# from keras.metrics import binary_accuracy
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

In [12]:
## 2 Read data and Rescale data



# dimensions of our images.
IM_HEIGHT, IM_WIDTH, IM_CHANNEL = 150, 150, 3
NB_EPOCHS = 100
BAT_SIZE = 32
FC_SIZE = 1024
    # data path
_data_path = './asset/data/'
train_data_dir = _data_path+"train/"
valid_data_dir = _data_path+"valid/"
test_data_dir = _data_path+"test/"

train_datagen = ImageDataGenerator(
        
        rescale=1. / 255,   # orignal image is 0~255
       
val_datagen = ImageDataGenerator(
        rescale=1. / 255,   
    )

test_datagen = ImageDataGenerator(
        rescale=1. / 255,   
    )
############################
## Insert your code here ###
############################

SyntaxError: invalid syntax (<ipython-input-12-f0e611ed6c33>, line 24)

In [13]:
## back up for read data from folds

# To set up the training and val directory
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(IM_HEIGHT, IM_WIDTH),
        batch_size=BAT_SIZE,
        class_mode="binary"
    )
val_generator = val_datagen.flow_from_directory(
        valid_data_dir,
        target_size=(IM_HEIGHT, IM_WIDTH),
        batch_size=BAT_SIZE,
        class_mode="binary"
    )
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(IM_HEIGHT, IM_WIDTH),
        batch_size=BAT_SIZE,
        class_mode="binary",
    )

Found 254 images belonging to 2 classes.
Found 65 images belonging to 2 classes.
Found 10 images belonging to 1 classes.


In [14]:
input_shape = (150, 150, 3)

In [15]:
## (2) build CNN model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras import optimizers

model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', 
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        9280      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)       147584    
__________

In [16]:
history = model.fit_generator(train_generator, steps_per_epoch=50, epochs=10,
                              validation_data=val_generator, validation_steps=50, verbose=1)

Epoch 1/10


ValueError: Error when checking input: expected conv2d_8_input to have shape (150, 150, 3) but got array with shape (32, 32, 3)

In [ ]:
## (3) Visualization 

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic CNN with butterfly dataset', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(0,10))
ax1.plot(epoch_list, history.history['acc'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 10, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 10, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
## (4) Test
# Evaluate the model on the test data using `evaluate`
pred = model.predict_generator(test_generator,verbose=1,steps=1)
print (pred)